In [1]:
import src.inference.helpers as ih
import src.models.builders as mb
import src.models.cpts.helpers as cpth
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import src.modelling_ar.ar as model_ar

In [2]:
(
    model,
    inf_alg,
    HFEV1,
    ecFEV1,
    AR,
    HO2Sat,
    O2SatFFA,
    IA,
    UO2Sat,
    O2Sat,
) = mb.o2sat_fev1_point_in_time_model_shared_healthy_vars(160, 30, "Male")

In [3]:
cpt = cpth.get_cpt([ecFEV1, HFEV1, AR])

## Find the prior that produces a uniform message up

In [42]:
ecFEV1.get_bins_arr()[0]

array([0.  , 0.05])

In [63]:
cpt_for_ecfev1 = cpt[ecFEV1.get_bin_for_value(1)[1], :, :]

ar_cpt = AR.cpt
# ar_cpt = 100000 + AR.midbins * AR.midbins * AR.midbins
# ar_cpt = AR.midbins + 50
# ar_cpt = np.exp(AR.midbins)
# ar_cpt = np.zeros(len(AR.midbins))
# ar_cpt[AR.get_bin_for_value(10)[1]] = 1

# ar_cpt = ar_fit
ar_cpt = ar_cpt / sum(ar_cpt)

hfev1 = np.matmul(cpt_for_ecfev1, ar_cpt) 

fig = make_subplots(rows=2, cols=1)
ih.plot_histogram(fig, AR, ar_cpt, AR.a, AR.b, 1, 1)
ih.plot_histogram(fig, HFEV1, hfev1, HFEV1.a, HFEV1.b, 2, 1)
fig.show()

In [64]:
cpt_hfev1 = hfev1
cpt_hfev1 = np.ones(len(HFEV1.midbins)) / len(HFEV1.midbins)
# cpt_hfev1 = HFEV1.midbins

ar = np.matmul(cpt_hfev1, cpt_for_ecfev1)

fig = make_subplots(rows=2, cols=1)
ih.plot_histogram(fig, HFEV1, cpt_hfev1, HFEV1.a, HFEV1.b, 1, 1)
ih.plot_histogram(fig, AR, ar, AR.a, AR.b, 2, 1)
fig.show()

In [65]:
fig = make_subplots(rows=1, cols=1)
# Add scatter plot
fig.add_trace(go.Scatter(x=AR.midbins, y=ar, mode="lines+markers", name="from hfev1"))
ar_tmp = np.log(AR.midbins) / sum(np.log(AR.midbins))
fig.add_trace(go.Scatter(x=AR.midbins, y=ar_tmp, mode="lines+markers", name="log fit"))
ar_tmp = AR.midbins * AR.midbins * AR.midbins + 100000
ar_tmp = ar_tmp / sum(ar_tmp)
fig.add_trace(
    go.Scatter(x=AR.midbins, y=ar_tmp, mode="lines+markers", name="square fit")
)
# Polyfit
ar_tmp = model_ar.get_prior_for_uniform_hfev1_message(AR)
fig.add_trace(go.Scatter(x=AR.midbins, y=ar_tmp, mode="lines+markers", name="poly fit"))
fig.show()

In [66]:
# Fit a polynomial function to the data in ar
for deg in [3, 12]:
    delete_last_idx=13
    print("degree", deg)
    p = np.polyfit(AR.midbins[:-delete_last_idx], ar[:-delete_last_idx], deg)
    ar_fit = np.polyval(p, AR.midbins)

    print("residual error", sum(np.abs(ar[:-delete_last_idx] - ar_fit[:-delete_last_idx])))

    message_up = np.matmul(cpt_for_ecfev1, ar_fit)

    fig = make_subplots(rows=1, cols=1)
    ih.plot_histogram(fig, HFEV1, message_up, HFEV1.a, HFEV1.b, 1, 1)
    fig.show()

# Plot the original data and the fitted data
fig = make_subplots(rows=1, cols=1)
# Add scatter plot
fig.add_trace(go.Scatter(x=AR.midbins, y=ar, mode="lines+markers", name="data"))
fig.add_trace(go.Scatter(x=AR.midbins, y=ar_fit, mode="lines+markers", name="fit"))
fig.show()

degree 3
residual error 0.002102777119490652


degree 12
residual error 5.196603262286326e-09


In [55]:
p

array([ 3.26421676e-24, -1.04268394e-21,  1.50839923e-19, -1.27449750e-17,
        6.97070555e-16, -2.54149246e-14,  6.45894402e-13, -9.68516684e-12,
        2.17787840e-10,  9.19676758e-09,  1.00333030e-06,  1.00004214e-04,
        1.00003368e-02])

### Solve matrix equation

In [76]:
np.linalg.inv(np.matmul(cpt_for_ecfev1.T, cpt_for_ecfev1))

LinAlgError: Singular matrix

In [98]:
cpt_for_ecfev1 = cpt[ecFEV1.get_bin_for_value(1)[1], :, :]

# b = CPT x X
# X = (CPT^T x CPT)^-1 x CPT^T x b

b = np.ones(HFEV1.card)/HFEV1.card

# Compute X -> can't do as the matrix is singular
# X = np.matmul(
#     np.matmul(
#         np.matmul(
#             np.linalg.inv(np.matmul(cpt_for_ecfev1.T, cpt_for_ecfev1)),
#             cpt_for_ecfev1.T,
#         ),
#         U,
#     )
# )

# Use SVD
U, S, Vh = np.linalg.svd(cpt_for_ecfev1)
# Now solve the equation
# X = np.matmul(np.matmul(np.matmul(Vh.T, np.linalg.inv(np.diag(S))), U.T), b)
X = np.matmul(Vh.T, np.linalg.inv(np.diag(S)))
X.shape

(45, 45)

In [102]:
print(cpt_for_ecfev1.shape)
np.linalg.matrix_rank(cpt_for_ecfev1)

(100, 45)


37

In [99]:
print(U.shape, S.shape, Vh.shape)

(100, 100) (45,) (45, 45)


### Check the resulting message up to HFEV1

In [23]:
prior = np.polyval(p, AR.midbins)

In [24]:
# prior = model_ar.get_prior_for_uniform_hfev1_message(AR)
message_up = np.matmul(cpt_for_ecfev1, prior)

fig = make_subplots(rows=1, cols=1)
ih.plot_histogram(fig, HFEV1, message_up, HFEV1.a, HFEV1.b, 1, 1)
fig.show()

# Is the prior uniform in log space?

In [85]:
ar_prior = model_ar.get_prior_for_uniform_hfev1_message(AR)

log_prior = np.log(1-ar_prior)
log_prior = log_prior / sum(log_prior)

fig = make_subplots(rows=1, cols=1)
# Add scatter plot
fig.add_trace(go.Scatter(x=100-AR.midbins, y=ar_prior, mode="lines+markers"))
# Add x axis title
fig.update_xaxes(title_text="1-AR: capacity to let air through")
fig.show()
